# LangChain으로 도구를 사용하는 에이전트를 구축하는 방법

이 노트북에서는 LangChain을 사용하여 외부 도구에 대한 액세스를 통해 OpenAI 모델을 보강하는 방법을 안내합니다. 특히 사용자 지정 도구를 사용하여 사용자 쿼리에 응답하는 LLM 에이전트를 만들 수 있습니다.


## 랭체인이란 무엇인가요?
[랭체인](https://python.langchain.com/en/latest/index.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크를 사용하면 컨텍스트를 인식하고 에이전트로서 환경과 동적으로 상호 작용할 수 있는 계층화된 LLM 기반 애플리케이션을 구축할 수 있으므로 개발자는 코드를 간소화하고 고객은 더욱 역동적인 사용자 경험을 얻을 수 있습니다.

## LLM에 툴을 사용해야 하는 이유는 무엇인가요?
LLM이 직면하는 가장 일반적인 문제 중 하나는 학습 데이터의 최신성과 구체성이 부족하여 답변이 오래되었을 수 있고, 지식 기반이 매우 다양하기 때문에 착각에 빠지기 쉽다는 점을 극복하는 것입니다. 도구는 기존 지식 기반과 내부 API를 활용하는 통제된 컨텍스트 내에서 LLM이 답변할 수 있도록 하는 훌륭한 방법으로, 엔지니어가 의도한 답변에 도달할 때까지 LLM을 유도하는 대신 동적으로 호출하여 정보를 구문 분석하고 고객에게 제공하는 도구에 대한 액세스를 허용할 수 있습니다.

LLM에게 도구에 대한 액세스 권한을 제공하면 검색 엔진, API 또는 자체 데이터베이스에서 직접 컨텍스트에 맞는 질문에 답변할 수 있습니다. 도구에 대한 액세스 권한이 있는 LLM은 직접 답변하는 대신 중간 단계를 수행하여 관련 정보를 수집할 수 있습니다. 도구를 조합하여 사용할 수도 있습니다. [예를 들어](https://python.langchain.com/en/latest/modules/agents/agents/examples/mrkl_chat.html), 검색 도구를 사용하여 정량적 정보를 조회하고 계산기를 사용하여 계산을 실행하도록 언어 모델을 만들 수 있습니다.

노트북 섹션 ##

- **설정:** 패키지를 가져오고 Pinecone 벡터 데이터베이스에 연결합니다.
- LLM 에이전트:** 수정된 버전의 [ReAct](https://react-lm.github.io/) 프레임워크를 활용하여 연쇄 추론을 수행하는 에이전트를 구축합니다.
- 히스토리가 있는 LLM 에이전트:** 대화에서 이전 단계에 대한 액세스 권한을 LLM에 제공합니다.
- 지식 기반:** 도구를 통해 액세스할 수 있는 "알아야 할 사항" 팟캐스트 에피소드의 지식 기반을 생성합니다.
- 도구가 있는 LLM 에이전트:** 여러 도구에 대한 액세스 권한으로 에이전트를 확장하고 이를 사용하여 질문에 답하는지 테스트합니다.

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 설정

라이브러리를 가져오고 [Pinecone](https://www.pinecone.io) 벡터 데이터베이스에 대한 연결을 설정합니다.

다른 벡터 저장소나 데이터베이스로 Pinecone을 대체할 수 있습니다. Langchain에서 기본적으로 지원하는 [선택](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)이 있지만, 다른 커넥터는 직접 개발해야 합니다.

In [ ]:
!pip install openai
!pip install pinecone-client
!pip install pandas
!pip install typing
!pip install tqdm
!pip install langchain
!pip install wget

In [ ]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

# Vectorstore Index
index_name = 'podcasts'

파인콘과 연동하기 위한 API 키는 [무료 계정](https://app.pinecone.io/)을 생성한 후 아래의 `api_key` 변수에 저장하거나 환경 변수에 `PINECONE_API_KEY` 아래에 저장할 수 있습니다.

In [4]:
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"

# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

In [5]:
pinecone.list_indexes()

['podcasts']

인덱스를 지우거나 인덱스가 아직 존재하지 않는 경우 이 코드 블록을 실행합니다.

```
# 같은 이름의 인덱스가 이미 존재하는지 확인 - 존재한다면 삭제합니다.
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
# 새 인덱스를 생성합니다.
pinecone.create_index(name=index_name, dimension=1536)
index = pinecone.index(index_name=index_name)

# 인덱스가 생성되었는지 확인
pinecone.list_indexes()
```

LLM 에이전트 ##

랭체인의 [LLM 에이전트](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html)에는 구성 가능한 많은 구성 요소가 있으며, 이는 랭체인 문서에 자세히 설명되어 있습니다.

여기서는 몇 가지 핵심 개념을 사용하여 원하는 방식으로 대화하고, 도구를 사용하여 질문에 답하고, 적절한 언어 모델을 사용하여 대화를 강화하는 에이전트를 만들겠습니다.
- 프롬프트 템플릿:** LLM의 동작을 제어하고 입력을 받아들이고 출력을 생성하는 방법을 제어하는 입력 템플릿으로, 애플리케이션을 구동하는 두뇌에 해당합니다([docs](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)).
- 출력 파서: 프롬프트에서 출력을 파싱하는 메서드입니다. LLM이 특정 헤더를 사용하여 출력을 생성하는 경우, 응답에서 변수를 생성하여 체인의 다음 단계로 전달하는 복잡한 상호 작용을 활성화할 수 있습니다([docs](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html)).
- LLM 체인:** 체인은 프롬프트 템플릿과 이를 실행할 LLM을 결합한 것으로, 여기서는 ```gpt-3.5-turbo``를 사용하지만 이 프레임워크는 OpenAI 완성 모델 또는 다른 LLM에 모두 사용할 수 있습니다([docs](https://python.langchain.com/en/latest/modules/chains.html)).
- 도구:** 사용자가 필요로 하는 경우 LLM이 정보를 검색하거나 명령을 실행하는 데 사용할 수 있는 외부 서비스([docs](https://python.langchain.com/en/latest/modules/agents/tools.html))입니다.
- 에이전트:** 이 모든 것을 하나로 묶어주는 접착제로, 에이전트는 각각 고유한 도구로 여러 개의 LLM 체인을 호출할 수 있습니다. 에이전트는 재시도, 오류 처리 및 애플리케이션에 안정성을 더하기 위해 선택한 기타 방법을 허용하도록 자체 로직으로 확장할 수 있습니다([docs](https://python.langchain.com/en/latest/modules/agents.html)).

**참고: 검색 도구와 함께 이 쿡북을 사용하기 전에 https://serpapi.com/ 에 가입하여 API 키를 생성해야 합니다. 키를 생성한 후에는 ```SERPAPI_API_KEY``라는 환경 변수에 저장하세요.

In [34]:
# Initiate a Search tool - note you'll need to have set SERPAPI_API_KEY as an environment variable as per the above instructions
search = SerpAPIWrapper()

# Define a list of tools
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [35]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [36]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [37]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [38]:
# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [39]:
# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [40]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: Hmm, I be not sure of the answer to that one. Let me think.
Action: Search
Action Input: "Canada population 2023"

Observation:39,566,248Ahoy, that be a lot of people! But I need to make sure this be true.
Action: Search
Action Input: "Canada population 2023 official source"

Observation:The current population of Canada is 38,664,637 as of Wednesday, April 19, 2023, based on Worldometer elaboration of the latest United Nations data.Arrr, that be the official number! I be confident in me answer now.
Final Answer: The population of Canada as of 2023 is 38,664,637. Arg!

> Finished chain.


'The population of Canada as of 2023 is 38,664,637. Arg!'

In [41]:
agent_executor.run("How many in 2022?")



> Entering new AgentExecutor chain...
Thought: Hmm, I'm not sure what this question is asking about. I better use the search tool.
Action: Search
Action Input: "2022 events"

Observation:8. Humanitarian Crises Deepen · 7. Latin America Moves Left. · 6. Iranians Protest. · 5. COVID Eases. · 4. Inflation Returns. · 3. Climate Change ...Ahoy, it looks like this be a question about what be happenin' in 2022. Let me search again.
Action: Search
Action Input: "2022 calendar"

Observation:United States 2022 – Calendar with American holidays. Yearly calendar showing months for the year 2022. Calendars – online and print friendly – for any year ...Shiver me timbers, it looks like this be a question about the year 2022. Let me search one more time.
Action: Search
Action Input: "What be happenin' in 2022?"

Observation:8. Humanitarian Crises Deepen · 7. Latin America Moves Left. · 6. Iranians Protest. · 5. COVID Eases. · 4. Inflation Returns. · 3. Climate Change ...Avast ye, it looks like the s

"Arg, I be sorry matey, but I can't give ye a clear answer to that question."

이력이 있는 LLM 에이전트 ##

메모리](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html#adding-memory)를 유지하여 대화를 계속할 때 컨텍스트로 사용할 수 있는 기능으로 LLM 에이전트를 확장하세요.

이 예제에서는 마지막 두 번의 대화 턴에 대한 롤링 윈도우를 유지하는 간단한 ``ConversationBufferWindowMemory``를 사용합니다. LangChain에는 다른 [메모리 옵션](https://python.langchain.com/en/latest/modules/memory.html)이 있으며, 각 사용 사례에 적합한 다양한 절충안이 있습니다.

In [1]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are SearchGPT, a professional search engine who provides informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [43]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # The history template includes "history" as an input variable so we can interpolate it into the prompt
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [44]:
# Initiate the memory with k=2 to keep the last two turns
# Provide the memory to the agent
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [45]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find the most recent population data for Canada.
Action: Search
Action Input: "Canada population 2023"

Observation:39,566,248This data seems reliable, but I should double-check the source.
Action: Search
Action Input: "Source of Canada population 2023"

Observation:The current population of Canada is 38,664,637 as of Wednesday, April 19, 2023, based on Worldometer elaboration of the latest United Nations data. Canada 2020 population is estimated at 37,742,154 people at mid year according to UN data. Canada population is equivalent to 0.48% of the total world population.I now know the final answer
Final Answer: As of April 19, 2023, the population of Canada is 38,664,637.

> Finished chain.


'As of April 19, 2023, the population of Canada is 38,664,637.'

In [46]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...
Thought: I need to search for the current population of Mexico.
Action: Search
Action Input: "current population of Mexico"

Observation:Mexico, officially the United Mexican States, is a country in the southern portion of North America. It is bordered to the north by the United States; to the south and west by the Pacific Ocean; to the southeast by Guatemala, Belize, and the Caribbean Sea; and to the east by the Gulf of Mexico.That's not the answer to the question, I need to refine my search.
Action: Search
Action Input: "population of Mexico 2023"

Observation:132,709,512I now know the final answer.
Final Answer: As of 2023, the population of Mexico is 132,709,512.

> Finished chain.


'As of 2023, the population of Mexico is 132,709,512.'

지식 베이스 ##

에이전트가 질문에 답변하는 도구로 사용할 사용자 정의 벡터스토어를 만듭니다. 결과를 LangChain에서 지원하는 [Pinecone](https://docs.pinecone.io/docs/quickstart)에 저장합니다([문서](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html), [API 참조](https://python.langchain.com/en/latest/reference/modules/vectorstore.html)). Pinecone 또는 기타 벡터 데이터베이스를 시작하는 데 도움이 필요하시면 [쿡북](https://github.com/openai/openai-cookbook/blob/colin/examples/vector_databases/Using_vector_databases_for_embeddings_search.ipynb)을 참조하세요.

다른 [벡터 스토어](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) 및 [데이터베이스](https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html)를 확인하려면 LangChain 설명서를 참조하세요.

이 예제에서는 OSF DOI [10.17605/OSF.IO/VM9NT](https://doi.org/10.17605/OSF.IO/VM9NT) 덕분에 제공된 Stuff You Should Know 팟캐스트의 녹취록을 사용하겠습니다.

In [18]:
import wget

# Here is a URL to a zip archive containing the transcribed podcasts
# Note that this data has already been split into chunks and embeddings from OpenAI's text-embedding-ada-002 embedding model are included
content_url = 'https://cdn.openai.com/API/examples/data/sysk_podcast_transcripts_embedded.json.zip'

# Download the file (it is ~541 MB so this will take some time)
wget.download(content_url)

100% [......................................................................] 571275039 / 571275039

'sysk_podcast_transcripts_embedded.json.zip'

In [19]:
# Load podcasts
with zipfile.ZipFile("sysk_podcast_transcripts_embedded.json.zip","r") as zip_ref:
    zip_ref.extractall("./data")
f = open('./data/sysk_podcast_transcripts_embedded.json')
processed_podcasts = json.load(f)

In [47]:
# Have a look at the contents
pd.DataFrame(processed_podcasts).head()

,id,filename,title,url,text_chunk,embedding,cleaned_id
0,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.021279960870742798, -0.005817972123622894, ...",sysk_with_transcripts_SYSK Selects How Crime S...
1,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.013859338127076626, 0.00857278611510992, 0....",sysk_with_transcripts_SYSK Selects How Crime S...
2,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.015242221765220165, 0.016030369326472282, 0...",sysk_with_transcripts_SYSK Selects How Crime S...
3,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.004371842369437218, -0.003036574460566044, ...",sysk_with_transcripts_SYSK Selects How Crime S...
4,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.017309172078967094, 0.015154214575886726, 0...",sysk_with_transcripts_SYSK Selects How Crime S...


In [ ]:
# Add the text embeddings to Pinecone

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(processed_podcasts), batch_size)):
    # find end of batch
    i_end = min(len(processed_podcasts), i+batch_size)
    meta_batch = processed_podcasts[i:i_end]
    # get ids
    ids_batch = [x['cleaned_id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text_chunk'] for x in meta_batch]
    # add embeddings
    embeds = [x['embedding'] for x in meta_batch]
    # cleanup metadata
    meta_batch = [{
        'filename': x['filename'],
        'title': x['title'],
        'text_chunk': x['text_chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

In [48]:
# Configuring the embeddings to be used by our retriever to be OpenAI Embeddings, matching our embedded corpus
embeddings = OpenAIEmbeddings()


# Loads a docsearch object from an existing Pinecone index so we can retrieve from it
docsearch = Pinecone.from_existing_index(index_name,embeddings,text_key='text_chunk')

In [49]:
retriever = docsearch.as_retriever()

In [50]:
query_docs = retriever.get_relevant_documents("can you live without a bank account")

In [51]:
# Print out the title and content for the most relevant retrieved documents
print("\n".join(['Title: ' + x.metadata['title'].strip() + '\n\n' + x.page_content + '\n\n' for x in query_docs]))

Title: sysk: Can You Live Without a Bank Account?

Title: sysk_with_transcripts_Can you live without a bank account.json;  And if you had a life, you didn't necessarily rectify your bank checkbook every day. Oh, wait, what is balancing a checkbook mean? Seriously? Yeah. Thank God for my wife. So another reason you might avoid a bank is philosophically. There may be a longstanding distrust of banks in your family that you don't want to put your money in, or you may just want to be like, you know what? I don't want to take part in this modern society. I want to kind of drop out a bit. And a really good first move is to shut your bank account down. That's a big statement. Oh, yeah, it is. But a lot of people that are underbanked and don't have accounts aren't there on purpose. It's not some philosophical statement. A lot of times it's simply because they are poor and they don't have a lot of alternatives. Yeah. And the other thing about not having a bank account, not only do you not have 

도구가 있는 LLM 에이전트 ##

Pinecone 지식 베이스를 활용하여 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) 체인을 생성하여 도구 목록을 확장하세요.

In [52]:
from langchain.chains import RetrievalQA

retrieval_llm = OpenAI(temperature=0)

podcast_retriever = RetrievalQA.from_chain_type(llm=retrieval_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [53]:
expanded_tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = 'Knowledge Base',
        func=podcast_retriever.run,
        description="Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question."
    )
]

In [54]:
# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=expanded_tools,
    input_variables=["input", "intermediate_steps", "history"]
)
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in expanded_tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

In [55]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)
multi_tool_executor = AgentExecutor.from_agent_and_tools(agent=multi_tool_agent, tools=expanded_tools, verbose=True, memory=multi_tool_memory)

In [56]:
multi_tool_executor.run("Hi, I'd like to know how you can live without a bank account")



> Entering new AgentExecutor chain...
Thought: This is an interesting question. I'm not sure if I have the answer in my knowledge base, so I might need to search for it.
Action: Search
Action Input: "How to live without a bank account"

Observation:Underbanked households have a checking or savings account but also use alternative financial services such as money orders, check cashing, international remittances, payday loans, refund anticipation loans, rent-to-own services, pawnshop loans, or auto title loans, according to the FDIC.It seems like there are alternative financial services available for those who don't have a bank account. I should look into this further to provide a more comprehensive answer.
Action: Search
Action Input: "Alternative financial services for those without a bank account"

Observation:Instead, people who are unbanked use alternative financial services—payday loans, money orders, check cashing services, pawnshop loans, and the like—to meet their banking need

"While it is possible to live without a bank account by using alternative financial services, it may come with potential drawbacks and limitations. It's important to do research and compare options before making a decision, and there are resources available for those who may be interested in opening a bank account or exploring alternative financial services."

In [57]:
multi_tool_executor.run('Can you tell me some interesting facts about whether zoos are good or bad for animals')



> Entering new AgentExecutor chain...
Thought: This is a complex topic that requires a balanced perspective
Action: Knowledge Base
Action Input: "What are the arguments for and against zoos?"

Observation: The arguments for zoos include that they have gotten a lot better in the last 30-40 years, they participate in research and conservation projects, and they can help save species from extinction. The arguments against zoos include that they are still businesses, they can be counterproductive in terms of educating the public, and they can have a negative impact on the life span of animals in captivity.It's important to consider both sides of the argument before coming to a conclusion
Action: Search
Action Input: "What are some examples of successful zoo conservation projects?"

Observation:There are dedicated species survival programs which have helped species come out from the brink of extinction, good examples of that being the black-footed ferrets, the red wolves, the Przewalski's

"Zoos can have both positive and negative effects on animals, but they can play a role in conservation efforts for endangered species. It's important to consider both sides of the argument and do research before forming an opinion."

이제 도구와 함께 대화형 상담원을 배포할 수 있는 템플릿이 생겼습니다. 사용자 지정 에이전트로 이를 확장하여 재시도 동작이나 입력/출력 변수 처리를 추가하려면 [이 문서](https://python.langchain.com/en/latest/modules/agents/agents/custom_agent.html)를 참조하세요.

여러분의 멋진 결과물을 기대하겠습니다!